In [1]:
%load_ext autoreload 
%autoreload 1
import os
os.environ['SNORKELDB'] = 'sqlite:///brat-import.db'
# from set_env import set_env
# set_env()
from snorkel import SnorkelSession
import snorkel.contrib.brat as brt
from snorkel.parsers import StanfordCoreNLPServer

from snorkel.models import Candidate, Document, candidate_subclass, GoldLabel

session = SnorkelSession()

In [2]:
corenlp_server = StanfordCoreNLPServer(version='3.6.0', split_newline=False, num_threads=1)

In [3]:
brat = brt.Brat(session)

In [4]:
# Only run on documents which have annotations

# from snorkel.parser import TextDocPreprocessor, CorpusParser
# import multiprocessing
# doc_preprocessor = TextDocPreprocessor("brat_data/test_labeled_docs/*.txt", encoding="utf-8")
# corpus_parser = CorpusParser()
# corpus_parser.apply(doc_preprocessor, parralelism=multiprocessing.cpu_count())

In [5]:
input_dir = "brat_data/test_labeled_docs/"
brat.import_project(input_dir, annotations_only=False, annotator_name='brat', num_threads=1, parser=corenlp_server)

In [6]:
pheno_spans = brat.explore()
print(len(pheno_spans))
print len([pheno_span for pheno_span in pheno_spans if len(pheno_span) > 1])

544
91


In [10]:
from snorkel.models import Document, Sentence, Span

In [11]:
PhenoPair = candidate_subclass('Phenotypes', ['descriptor', 'entity'])

/home/pyke/workspace/snorkel/venv/local/lib/python2.7/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as snorkel.models.candidate.Phenotypes, and will be replaced in the string-lookup table.
  item.__name__


InvalidRequestError: Table 'phenotypes' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [ ]:
# GenePhenoPair = candiate_subclass('GenePhenoPair', )

In [17]:
import collections
from sqlalchemy import and_

jaccard_cutoff = 0.5

candidates = session.query(PhenoPair).filter(PhenoPair.split == 0).all()
print(len(candidates), "candidates")
sentence_to_pheno = collections.defaultdict(list)

for pheno_span in pheno_spans:
    # pheno_span is a list of fragments, where each fragment is a temporary span
    sent_id = pheno_span[0].sentence.id
    sentence_to_pheno[sent_id].append(pheno_span)

num = 0
num_matches = 0
num_sentences = 0
num_gold_phenos = 0
num_zero_matches = 0
avg = 0
for sentence_id, gold_phenos in sentence_to_pheno.iteritems():
    sentence_match = False
    results = session.query(Span, PhenoPair).filter(and_(Span.sentence_id == sentence_id, PhenoPair.descriptor_id == Span.id)).all()
    if (len(results) == 0):
        num_zero_matches += 1
#         print "sentence", sentence_id
#         for gold_pheno in gold_phenos:
#             print(gold_pheno.get_span())
#         print
        
    for gold_pheno_fragments in gold_phenos:
        matched = False
        for span, cand_pheno in results:
            # Build word indice set for each of the gold phenotype, and the candiate phenotype
            cand_words = set()
            gold_words = set()
            # combine both entities (modifier and attribute) into one "phenotype"
            [cand_words.update(xrange(span.get_word_start(), span.get_word_end()+1)) for span in cand_pheno.get_contexts()]
            
            for gold_pheno in gold_pheno_fragments:
                gold_words.update(xrange(gold_pheno.get_word_start(), gold_pheno.get_word_end()+1))
            intersect = gold_words.intersection(cand_words)
            jaccard_score = float(len(intersect)) / len(cand_words.union(gold_words))
            
            
            if jaccard_score > jaccard_cutoff:
                matched = True
                sentence_match = True
                avg += jaccard_score
                num += 1
                print(jaccard_score)
                print(gold_pheno_fragments)
                print(cand_pheno)
#             print(cand_pheno)
                print("\n")
        num_gold_phenos += 1
        if matched:
            num_matches += 1
    if sentence_match:
        num_sentences += 1

print("avg", avg / num)
recall = float(num_matches) / num_gold_phenos
print("num gold phenos", num_gold_phenos)
num_sentences_tagged = len(sentence_to_pheno.keys())
print("num_sentences_tagged", num_sentences_tagged)
print("sentences", float(num_sentences) / num_sentences_tagged)
print("num sents missed", float(num_zero_matches) / num_sentences_tagged)
print("recall", recall)

(0, 'candidates')


ZeroDivisionError: integer division or modulo by zero

In [9]:
# each entity/relation type is assigned to a different split
#for i in range(0,1):

print len(candidates)
for i,c in enumerate(candidates):
    print type(c).type, c
    if i > 5:
        break
print
    
# ProteinComponent relations
candidates = session.query(PhenoPair).filter(PhenoPair.split == 0).all()

42972
Phenotypes.type Phenotypes(Span("First-order rosette branch number was also slightly reduced in", sentence=405564, chars=[0,61], words=[0,8]), Span("also slightly reduced in quadruple mutants compared to wild type", sentence=405564, chars=[38,101], words=[5,14]))
Phenotypes.type Phenotypes(Span("First-order rosette branch number was also slightly reduced in", sentence=405564, chars=[0,61], words=[0,8]), Span("first-order rosette branches", sentence=405564, chars=[120,147], words=[21,23]))
Phenotypes.type Phenotypes(Span("First-order rosette branch number was also slightly reduced in", sentence=405564, chars=[0,61], words=[0,8]), Span("First-order rosette branch number", sentence=405564, chars=[0,32], words=[0,3]))
Phenotypes.type Phenotypes(Span("branches were generated much more slowly than the wild", sentence=405564, chars=[140,193], words=[23,31]), Span("also slightly reduced in quadruple mutants compared to wild type", sentence=405564, chars=[38,101], words=[5,14]))
Phenotype

In [7]:
from snorkel.viewer import SentenceNgramViewer

sv = SentenceNgramViewer(candidates, session=session, n_per_page=6, height=400,
                         annotator_name='brat')
sv

<IPython.core.display.Javascript object>

In [ ]:
# Select spans from brat
# Select candidates from phenos
# For each candidate in phenos
# 